<a href="https://colab.research.google.com/github/lucyvost/dtc_docking/blob/main/DTC_smolecules_day_2_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2: virtual screening with gnina

gnina is scoring function consisting of an ensemble of convolutional neural networks (CNNs). In this section of the workshop we're going to be docking some of the molecules generated with DiffSBDD, and then carrying out some common sense tests to see if gnina can identify poses that classically would fail.

again, make sure to change runtime to GPU before you run anything!

first, we'll sort our installs...

In [1]:
%%capture
!wget https://github.com/gnina/gnina/releases/download/v1.0.3/gnina
!obrms -f lig.pdb  docked.sdf
!chmod +x gnina
!apt install openbabel
!pip install rdkit
!pip install molgrid


then we will download the protein target we're looking at and split it into receptor and ligand for docking:

In [ ]:
!wget http://files.rcsb.org/download/3RFM.pdb
!grep ATOM 3RFM.pdb > rec.pdb
!grep HETATM 3RFM.pdb > lig.pdb

Now let's dock the known ligand back into the empty pocket, and score the docks!

In [ ]:
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb -o docked.sdf --seed 0 > output.txt
gnina_results = {'mode':[], 'affinity': [], 'cnn pose score':[ ], 'cnn affinity': []}
with open( f"output.txt", "r") as logfile:

  lines = logfile.readlines()
  index_of_results = next((index for index, line in enumerate(lines) if line.startswith('mode')), None) + 3

  for n in range(5):

    gnina_results['cnn affinity'].append(float(lines[index_of_results+n][36:41]))
    gnina_results['cnn pose score'].append(float(lines[index_of_results + n][24:29]))
    gnina_results['affinity'].append(float(lines[index_of_results+ n][11:17]))
    gnina_results['mode'].append(float(lines[index_of_results + n][4]))

24


In [ ]:
import pandas as pd
pd.DataFrame(gnina_results)

,mode,affinity,cnn pose score,cnn affinity
0,1.0,-5.27,0.721,4.699
1,2.0,-4.97,0.709,4.676
2,3.0,-5.13,0.708,4.736
3,4.0,-5.05,0.704,4.615
4,5.0,-4.94,0.651,4.517


Plot the distribution of the predicted CNN affinity and pose score. Do they vary roughly how you'd expect them to?

In [ ]:
from matplotlib import pyplot as plt
#plot the distributions here - you can even do some funky stats on them if you so desire

Now, we're going to see how many of the interactions that the native ligand makes are conserved. You did this by eye in the first part of the workshop, but now we're going to do it with rdkit.
Hints:


*   Using pymol, find the coordinates of the ligand atoms that form bonds with protein atoms (might just be one for hydrogen)
*   Iterate through the atoms in the ligand and see if there are any of the same type as the ones that form the bonds in the crystal structure (you can find this out using the code below). If there are, calculate how far away they are from the coordinates of the crystal ligand's bond-forming atom(s)





In [ ]:
#Create feature factory
import os
import numpy as np
from rdkit import RDConfig
from rdkit.Chem import ChemicalFeatures
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)


#position of the bond-forming atom, found in pymol:
crystal_pos = [0,0,0]


mol = Chem.SDMolSupplier('docked.sdf')[0] # looking at the first dock only
feats = factory.GetFeaturesForMol(mol)

dict_pharm_type_pos = {'type':[],'distance from original':[] }
for feat in feats:
  pharmPosition = np.array(mol.GetConformer().GetAtomPosition(feat.GetAtomIds()[0]))
  dict_pharm_type_pos['type'].append(feat.GetFamily())
  dict_pharm_type_pos['distance from original'].append( np.linalg.norm(pharmPosition - crystal_pos))

pd.DataFrame(dict_pharm_type_pos)

,type,distance from original
0,Donor,45.831378
1,Donor,46.514077
2,Donor,49.479482
3,Acceptor,44.212638
4,Acceptor,47.523592
5,PosIonizable,49.479482


Upload one of the sdfs files that you generated in part 1 of the workshop, and repeat the above steps. Remember to make sure your files are in a format that gnina can understand!



In [ ]:
#docking generated poses

Go back to the first notebook and pick any other protein target to generate compounds for, making sure you use the same sampling settings as the ones you used for the 3RFM compounds you've uploaded here. Then, swap the proteins round and dock the generated compounds for each protein in the other's receptor - are the scores, on average, any lower? what if you dock the native ligands into the other's receptors? based on this..


*   How much structural information do you think that DiffSBDD managed to incorporate in the generated compounds?
*   How accurate do you think docking and scoring with gnina is?



Now, we're going to try and do some ridiculous stuff to our generated compounds and see if we can get gnina to break. Try calculating the centre of mass of the protein, and shifting each of the ligands towards it (simulating a clash) - can gnina still score them? How do they do?

In [ ]:
#calculate CoM of protein
#iterate over the coordinates of the ligand atoms and shift each one by 5A into the protein
#score with gnina